# Multispectral Remote Sensing Data: Urban Green Space

Visualize and quantify differences in vegetation health by neighborhood in Chicago, IL.

## STEP 1: Set up

### Package imports

In [2]:
import os
import shutil
from glob import glob

import earthpy as et
import earthpy.earthexplorer as etee
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import numpy as np
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrmerge

data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
chi_dir = os.path.join(data_dir, 'chicago_neighborhoods')
ndvi_dir = os.path.join(data_dir, 'chicago-green-space', 'processed')

for a_dir in [chi_dir, ndvi_dir]:
    if not os.path.exists(a_dir):
            os.makedirs(a_dir)

## STEP 2: Area of Interest

### Select Humboldt Park and Lincoln Park neighborhoods to test code and loops

Download a shapefile of the City of Chicago neighborhoods from [the City of Chicago Data Portal](https://data.cityofchicago.org/).



In [3]:
chi_path = os.path.join(chi_dir, 'chicago-neighborhoods.shp')
if not os.path.exists(chi_path):
    chi_url = (
        "https://data.cityofchicago.org/api/geospatial/"
        "bbvz-uum9?method=export&format=Shapefile")
    gpd.read_file(chi_url).to_file(chi_path)
    
chi_gdf = gpd.read_file(chi_path).set_index('pri_neigh')   

neigh_gdf = (
    chi_gdf
    .loc[['Humboldt Park', 'Lincoln Park']]
)
neigh_gdf

,sec_neigh,shape_area,shape_len,geometry
pri_neigh,,,,
Humboldt Park,HUMBOLDT PARK,1.250104e+08,46126.751351,"POLYGON ((-87.74060 41.88782, -87.74060 41.887..."
Lincoln Park,LINCOLN PARK,6.740145e+07,61856.516472,"POLYGON ((-87.62640 41.91137, -87.62646 41.911..."


## STEP 3: Download and process raster data

Convert the operations from loops into a **function** to make the code more efficient. 

In [3]:
def download_neighborhood_data(name, geometry, start, end):
    """
    Download NAIP raster for a given geometry, start date, and end date
    
    Downloads data from the National Agricultural Imagery Program (NAIP)
    given a spatial and temporal extent.
    <citation>

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """
    
    print(f'\nNeighborhood name: {name}')
    bbox = etee.BBox(*geometry.bounds)
    naip_downloader = etee.EarthExplorerDownloader(
        dataset="NAIP", 
        label=name.lower().replace(' ', '-'),
        bbox=bbox,
        start=start, 
        end=end,
        store_credential=True)
    naip_downloader.submit_download_request()
    naip_downloader.download(override=False)
    return naip_downloader

ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')
if os.path.exists(ndvi_stats_path):
    print('Reading in NDVI statistics file...')
    ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col='neighborhood')
else:
    print('NDVI statistics file does not exist...')
    # initialize a new, empty dataframe if doesnt exist
    ndvi_stats_df = pd.DataFrame()
for neighborhood_name, details in neigh_gdf.iterrows():
    if neighborhood_name in ndvi_stats_df.index:
        print('Neighborhood stats have already been calculated. Skipping')
        continue
        
    downloader = download_neighborhood_data(
        neighborhood_name, details.geometry, '2021-01-01', '2021-12-31')

Reading in NDVI statistics file...
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping


Write a function for the loop that loads and merges the arrays.

In [4]:
def load_and_merge_arrays(name):
    """
    Load in and merge downloaded arrays

    Parameters
    ==========
    name : str
      The name used to label the download
      

    Returns
    =======
    merge_da : rxr.DataArray
      DataArray with the merged data
    """
    
    print(f'\nNeighborhood name: {name}')
    data_path = os.path.join(
        et.io.HOME, et.io.DATA_NAME, 
        name.lower().replace(' ', '-'))
    tif_paths = glob(os.path.join(data_path, '*.tif'))
    das = [rxr.open_rasterio(tp, masked = True) for tp in tif_paths]
    merged_da = rxrmerge.merge_arrays(das)
    return merged_da

# Run to test functions
# merged_da = load_and_merge_arrays('Lincoln Park')
# merged_da

Write a function that computes the NDVI summary statistics and adds them to the statistics file (if the statistics are not already present).

In [4]:
def calculate_ndvi_statistics(gdf, da, stats_path, override=False):
    """
    Calculate NDVI, then summarize and save statistics
    
    Uses downloaded National Agricultural Imagery Program (NAIP)
    data. <citation>

    Parameters
    ==========
    gdf : gpd.GeoDataFrame
        One row with the neighborhood name and boundary
    da : rxr.DataArray
        Multispectral (NAIP) raster data
    stats_path : pathlike
        The path to the statistics file to save results
    """
    name = str(gdf.index[0])
    print(f'Neighborhood name: {name}')
    
    file_is_empty = True
    if os.path.exists(stats_path):
        print('Stats file exists.')
        stats_df = pd.read_csv(stats_path)
        file_is_empty = len(stats_df)==0
        print(f'Stats file is empty? {file_is_empty}')
        
        if not file_is_empty:
            print(name)
            print(list(stats_df.neighborhood))
            print(name in list(stats_df.neighborhood))
            if (name in list(stats_df.neighborhood)) and (not override):
                print('Stats already calculated. Skipping...')
                return
    
    reprojected_gdf = gdf.to_crs(da.rio.crs)

    naip_crop_da = da.rio.clip_box(*reprojected_gdf.total_bounds)
    naip_da = naip_crop_da.rio.clip(reprojected_gdf.geometry)
    
    ndvi_da = (
        (da.sel(band=4) - da.sel(band=1))
        / (da.sel(band=4) + da.sel(band=1))
    )
    
    print('Writing stats to file.')
    mode = 'w' if file_is_empty else 'a'
    pd.DataFrame(dict(
        neighborhood=[name],
        ndvi_25pctl=[np.nanpercentile(ndvi_da, 25)],
        ndvi_mean=[float(ndvi_da.mean())],
        ndvi_75pctl=[np.nanpercentile(ndvi_da, 75)],
        ndvi_min=[float(ndvi_da.min())],
        ndvi_max=[float(ndvi_da.max())],
        ndvi_median=[np.nanmedian(ndvi_da)],
        ndvi_std=[np.nanstd(ndvi_da)]
    )).to_csv(stats_path, mode=mode, header=file_is_empty, index=False)

# Run to test functions    
# calculate_ndvi_statistics(
#     chi_gdf.loc[['Lincoln Park']], merged_da, ndvi_stats_path)

I was partially succesful to complete the final steps. I created a loop that started off with just the two neighborhood `GeoDataFrame` (neigh_gdf) that I later replaced for the enitre chi_gdf. I run each of the functions in the loop, checking that they work.
Finally, I was supposed to write a line of code at the end my loop to delete the raster data files once I have saved the statistics I needed. Unfortunately, I could not use the `shutil.rmtree()` function becasue I kept getting a PermissionError. Below, I commented out some code that seemed promising but did not solve my issue. 

In [6]:
ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')
# import time
    
for neighborhood_name, details in neigh_gdf.iterrows():
    if not os.path.exists(ndvi_stats_path):
        print('NDVI statistics file does not exist...')
        ndvi_stats_df = pd.DataFrame()
    else:
        ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
    
    if neighborhood_name in ndvi_stats_df.index:
        print('Neighborhood stats have already been calculated. Skipping')
        continue
    
    # try:
    #     downloader = download_neighborhood_data(
    #         neighborhood_name, details.geometry, '2021-01-01', '2021-12-31')
    #     merged_da = load_and_merge_arrays(neighborhood_name)
    #     calculate_ndvi_statistics(
    #         chi_gdf.loc[[neighborhood_name]], merged_da, ndvi_stats_path)
    # except PermissionError as e:
    #     print(f"PermissionError: {e}")
    #     print(f"The file causing the issue: {downloader.data_dir}")
    # finally:
    #     # Introduce a 2 s delay before attempting to delete the directory
    #     time.sleep(2)
    #     if os.path.exists(downloader.data_dir):
    #         shutil.rmtree(downloader.data_dir)
    
    downloader = download_neighborhood_data(
        neighborhood_name, details.geometry, '2021-01-01', '2021-12-31')
    merged_da = load_and_merge_arrays(neighborhood_name)
    calculate_ndvi_statistics(
        chi_gdf.loc[[neighborhood_name]], merged_da, ndvi_stats_path)
        
    # shutil.rmtree(downloader.data_dir)
    

Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping


## STEP 4: Plotting NDVI statistics for available neighborhoods 

In [18]:
ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
chloropleth_mean = gv.tile_sources.EsriImagery * gv.Polygons(
    chi_gdf.join(ndvi_stats_df, how='left'),
    vdims=['ndvi_mean']
).opts(axiswise='bare',
       show_frame=False,
       xaxis=None,
       yaxis=None,
       cmap='RdYlGn',
       colorbar=True,
       clim=(ndvi_stats_df['ndvi_mean'].min(), ndvi_stats_df['ndvi_mean'].max()),
       tools=['hover'],
       title='Mean NDVI in Chicago\nneighborhoods'
)

hv.save(chloropleth_mean, 'chloropleth_mean.html')

# Convert 'ndvi_75pct' and 'ndvi_25pct' to numeric
ndvi_stats_df['ndvi_75pctl'] = pd.to_numeric(ndvi_stats_df['ndvi_75pctl'], errors='coerce')
ndvi_stats_df['ndvi_25pctl'] = pd.to_numeric(ndvi_stats_df['ndvi_25pctl'], errors='coerce')

# Calculate the difference between 'ndvi_75pct' and 'ndvi_25pct'
ndvi_stats_df['ndvi_iqr'] = ndvi_stats_df['ndvi_75pctl'] - ndvi_stats_df['ndvi_25pctl']

min_iqr_value = ndvi_stats_df['ndvi_iqr'].min()
max_iqr_value = ndvi_stats_df['ndvi_iqr'].max()

# Create chloropleth_iqr plot
chloropleth_iqr = gv.tile_sources.EsriImagery * gv.Polygons(
    chi_gdf.join(ndvi_stats_df, how='left'),
    vdims=['ndvi_iqr']
).opts(axiswise='bare',
       show_frame=False,
       xaxis=None,
       yaxis=None,
       cmap='oranges',
       colorbar=True,
       clim=(min_iqr_value, max_iqr_value),
       tools=['hover'],
       title='NDVI variations in Chicago\nneighborhoods (based on IQR)'
)

       
hv.save(chloropleth_iqr, 'chloropleth_iqr.html')

#### Based on the limited availability of neighborhood statistics the mean and interquartile range are not as meaningful as if we were able to compare more polygons with calculated statistics. The interquartile range was meant to illustrate a measure of spread in NDVI values among neighborhoods.

In [20]:
%%capture
%%bash
jupyter nbconvert multispectral-functions.ipynb --to html --no-input